# Claw Contraption

In [104]:
import numpy as np
import re

## Part 1

In [105]:
def parse_input(sample=True, file_path="Claw_Contraption.txt", mode="r"):
	if sample:
		file_path = "sample.txt"

	with open(file_path, mode) as file:
		data = file.read().strip()

	blocks = data.split("\n\n")

	return blocks

In [106]:
# Added offset for part 2
def extract_X_Y_Prize(blocks, offset=0):
	
	pattern = (
        r"Button A: X\+(\d+), Y\+(\d+)\n"
        r"Button B: X\+(\d+), Y\+(\d+)\n"
        r"Prize: X=(\d+), Y=(\d+)"
    )
	
	machines = []

	for block in blocks:
		match = re.match(pattern, block)
		if match:
			values = {
				"AX": int(match.group(1)), 
				"AY": int(match.group(2)),
				"BX": int(match.group(3)), 
		  		"BY": int(match.group(4)),
				"PX": int(match.group(5)) + offset,
				"PY": int(match.group(6)) + offset,
			}

			machines.append(values)

	return machines

### Notes

Equation setup:

- Let $a$ be the number of presses of button A
- Let $b$ be the number of presses of button B

\begin{align*}

	AX\cdot a + BX\cdot b &= PX \\
	AY\cdot a + BY\cdot b &= PY

\end{align*}

In matrix form:

\begin{align*}

	C \cdot X = P \equiv 

	\begin{bmatrix}
		AX & BX \\
		AY & BY
	\end{bmatrix}
	\begin{bmatrix}
		a \\
		b
	\end{bmatrix}
	&=
	\begin{bmatrix}
		PX \\
		PY
	\end{bmatrix}

\end{align*}

Using Cramer's rule:

\begin{align*}

	det(C) &= 
	\left | 
		\begin{matrix}
			AX & BX \\
			AY & BY
		\end{matrix}
	\right | \\

	det(C_a) &=
	\left | 
		\begin{matrix}
			PX & BX \\
			PY & BY
		\end{matrix}
	\right | \\

	det(C_b) &=
	\left | 
		\begin{matrix}
			AX & PX \\
			AY & PY
		\end{matrix}
	\right |
	
\end{align*}

Solving:

\begin{align*}

	a &= \frac{det(C_a)}{det(C)} \\
	b &= \frac{det(C_b)}{det(C)}

\end{align*}

In [107]:
def solve_cramers(machines):
	results = []
	for machine in machines:
		det = machine["AX"] * machine["BY"] - machine["BX"] * machine["AY"]
		det_a = machine["PX"] * machine["BY"] - machine["BX"] * machine["PY"]
		det_b = machine["AX"] * machine["PY"] - machine["PX"] * machine["AY"]

		a = det_a / det
		b = det_b / det

		results.append((a, b))

	return results

In [108]:
def valid_solution(results, one_hundred_rule=True, tol=1e-6):
	valid_results = []
	for (a, b) in results:
		if one_hundred_rule and (a > 100 or b > 100):
			continue
		
		if a - int(a) <= tol and b - int(b) <= tol:
			a = int(a)
			b = int(b)

			valid_results.append((a, b))

	return valid_results

In [109]:
def calculate_token(valid_results, A_price=3, B_price=1):
	tokens = []
	for (a, b) in valid_results:
		tokens.append(a * A_price + b * B_price)

	return tokens

In [110]:
def part1(sample=True):
	data = parse_input(sample)
	machines = extract_X_Y_Prize(data)
	results = solve_cramers(machines)
	valid_results = valid_solution(results)
	tokens = calculate_token(valid_results)

	return sum(tokens)

In [111]:
part1(sample=True), part1(sample=False)

(480, 28262)

## Part 2

In [112]:
def part2(sample=True, offset=10000000000000):
	data = parse_input(sample)
	machines = extract_X_Y_Prize(data, offset)
	results = solve_cramers(machines)
	valid_results = valid_solution(results, one_hundred_rule=False)
	tokens = calculate_token(valid_results)

	return sum(tokens)

In [113]:
part2(sample=True), part2(sample=False)

(875318608908, 101406661266314)